In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import the libraries

In [85]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [86]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load the dataset

In [87]:
df=pd.read_csv("/kaggle/input/sms-spam-collection/SpamCollectionSMS.txt",header=None,sep='\t',names=['label','message'])

In [88]:
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ã¼ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


## Basic information about the dataset

In [89]:
df.shape

(5572, 2)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [91]:
df.isnull().sum()

label      0
message    0
dtype: int64

In [92]:
df.nunique()

label         2
message    5169
dtype: int64

In [93]:
df.drop_duplicates(subset='message')

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ã¼ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


## Data Preprocessing

In [94]:
ps=PorterStemmer()
ps

<PorterStemmer>

In [95]:
corpus=[]

In [96]:
for i in range(0,len(df)):
    review=re.sub('[^a-zA-Z]',' ',df['message'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

## Creating the bag of words model

In [97]:
cv=CountVectorizer()
cv

CountVectorizer()

In [98]:
X=cv.fit_transform(corpus).toarray()
y=pd.get_dummies(df['label'])

In [99]:
y=y.iloc[:,1].values

## Split into training and testing sets

In [100]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

## train the model using Naive bayes clasifier

In [101]:
model=MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [102]:
y_pred=model.predict(X_test)
y_pred

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

In [103]:
cm=confusion_matrix(y_test,y_pred)
cm

array([[941,  16],
       [  7, 151]])

In [104]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.979372197309417

In [105]:
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.99      0.98      0.99       957
           1       0.90      0.96      0.93       158

    accuracy                           0.98      1115
   macro avg       0.95      0.97      0.96      1115
weighted avg       0.98      0.98      0.98      1115

